In [ ]:
import pickle
from google.colab import drive

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import os

In [ ]:
# Google dirve에 연결

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PATH
PATH_train='/content/drive/MyDrive/montage_project/train/'
PATH_train_montage = '/content/drive/MyDrive/montage_project/train/montage/'
PATH_train_json = '/content/drive/MyDrive/montage_project/train/H/'

PATH_train_resized='/content/drive/MyDrive/montage_project/train/resized/'

In [ ]:
PATH_test='/content/drive/MyDrive/montage_project/test/'
PATH_test_montage = '/content/drive/MyDrive/montage_project/test/montage/'
PATH_test_json = '/content/drive/MyDrive/montage_project/test/H/'


In [ ]:
# 라벨 설정
class_names_age = ['20~35','25~50','50']
class_names_gender = ['M','F']
class_names_face = ['계란형','긴형','둥근형','마름모형','사각형','역삼각형']
class_names_hairstyle = ['곱슬머리','직모(생머리)','나머지']
class_names_eyebrows =['각진형','둥근형','물결형','직선형','분류없음']

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 불러오기

In [ ]:
import gzip
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강

## train 불러오기!
with gzip.open(PATH_train+'dic_train_label2.pickle','rb') as f:
  dic_train_label = pickle.load(f)

print(len(dic_train_label),dic_train_label.keys())

6 dict_keys(['train_img_ar', 'train_age_label', 'train_gender_label', 'train_face_label', 'train_hairstyle_label', 'train_eyebrows_label'])


In [ ]:
dic_train_label['train_img_ar'][0].shape

(224, 224, 3)

In [ ]:
len(dic_train_label['train_img_ar']),len(dic_train_label['train_age_label']),len(dic_train_label['train_gender_label']),len(dic_train_label['train_hairstyle_label'])

(32089, 32089, 32089, 32089)

In [ ]:
import gzip
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강

## test 불러오기!
with gzip.open(PATH_test+'dic_test_label2.pickle','rb') as f:
  dic_test_label = pickle.load(f)

print(len(dic_test_label),dic_test_label.keys())

6 dict_keys(['test_img_ar', 'test_age_label', 'test_gender_label', 'test_face_label', 'test_hairstyle_label', 'test_eyebrows_label'])


In [ ]:
dic_test_label['test_img_ar'][0].shape

(224, 224, 3)

In [ ]:
len(dic_test_label['test_img_ar']),len(dic_test_label['test_age_label']),len(dic_test_label['test_gender_label']),len(dic_test_label['test_hairstyle_label'])

(1010, 1010, 1010, 1010)

# RESNET50 - hairstyle


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
type(dic_train_label['train_img_ar']),type(dic_test_label['test_img_ar'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_img_ar']),len(dic_test_label['test_img_ar'])

(32089, 1010)

In [ ]:
dic_train_label['train_img_ar'][0].shape,dic_test_label['test_img_ar'][0].shape

((224, 224, 3), (224, 224, 3))

In [ ]:
type(dic_train_label['train_hairstyle_label']),type(dic_test_label['test_hairstyle_label'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_hairstyle_label']),len(dic_test_label['test_hairstyle_label'])

(32089, 1010)

In [ ]:
dic_train_label['train_hairstyle_label'][10000],dic_test_label['test_hairstyle_label'][1000]

(0, 1)

In [ ]:
set(dic_train_label['train_hairstyle_label'])

{0, 1, 2}

In [ ]:
train_input = dic_train_label['train_img_ar']
test_input = dic_test_label['test_img_ar']

train_target = dic_train_label['train_hairstyle_label']
test_target =dic_test_label['test_hairstyle_label']


from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input,
                                                                      train_target,
                                                                      shuffle=True, 
                                                                      stratify=train_target, 
                                                                      random_state=34)

In [ ]:
train_input = train_input.astype('float32')
val_input=val_input.astype('float32')
test_input=test_input.astype('float32')

In [ ]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0)

In [ ]:
zero_mean_img = zero_mean(train_input)
zero_mean_img.shape

(224, 224, 3)

In [ ]:
train_input -= zero_mean_img
val_input -= zero_mean_img
test_input -= zero_mean_img

In [ ]:
train_input.shape,train_target.shape,val_input.shape,val_target.shape

((24066, 224, 224, 3), (24066,), (8023, 224, 224, 3), (8023,))

In [ ]:
test_input.shape,test_target.shape

((1010, 224, 224, 3), (1010,))

In [ ]:
set(train_target)

{0, 1, 2}

In [ ]:
## 일부 재학습 모델
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 49,278,851
Trainable params: 30,156,803
Non-tra

In [ ]:
# 학습
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강(train = 32089개) + zeromean 불가능

model.fit(train_input, train_target, epochs = 15, batch_size= 125, validation_data=(val_input, val_target))

Epoch 1/15
193/193 [==============================] - 2805s 15s/step - loss: 1.0239 - accuracy: 0.5739 - val_loss: 0.7809 - val_accuracy: 0.6510 - lr: 1.0000e-04
Epoch 2/15
193/193 [==============================] - 2769s 14s/step - loss: 0.7659 - accuracy: 0.6582 - val_loss: 0.7508 - val_accuracy: 0.6677 - lr: 1.0000e-04
Epoch 3/15
193/193 [==============================] - 2793s 14s/step - loss: 0.6896 - accuracy: 0.6952 - val_loss: 0.7360 - val_accuracy: 0.6759 - lr: 1.0000e-05
Epoch 4/15
193/193 [==============================] - 2812s 15s/step - loss: 0.6638 - accuracy: 0.7073 - val_loss: 0.7343 - val_accuracy: 0.6785 - lr: 1.0000e-06
Epoch 5/15
193/193 [==============================] - 2825s 15s/step - loss: 0.6601 - accuracy: 0.7068 - val_loss: 0.7344 - val_accuracy: 0.6778 - lr: 1.0000e-07
Epoch 6/15
193/193 [==============================] - 2834s 15s/step - loss: 0.6596 - accuracy: 0.7087 - val_loss: 0.7344 - val_accuracy: 0.6777 - lr: 1.0000e-08
Epoch 7/15
193/193 [========

In [ ]:
## fine-tune
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_input, train_target, epochs = 15, validation_data=(val_input, val_target), 
                   batch_size= 125)

Epoch 1/15
193/193 [==============================] - 8203s 42s/step - loss: 1.0393 - accuracy: 0.5492 - val_loss: 0.9514 - val_accuracy: 0.5529
Epoch 2/15
193/193 [==============================] - 8236s 43s/step - loss: 0.8995 - accuracy: 0.5887 - val_loss: 0.8289 - val_accuracy: 0.6373
Epoch 3/15
193/193 [==============================] - 8200s 42s/step - loss: 0.7788 - accuracy: 0.6563 - val_loss: 0.7830 - val_accuracy: 0.6658
Epoch 4/15
 89/193 [============>.................] - ETA: 1:06:44 - loss: 0.7319 - accuracy: 0.6840

In [ ]:
# 시험
model.evaluate(test_input, test_target)